# How to connect to OpenAI service via PromptSail and OpenAI Python SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [38]:
from openai import OpenAI
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(
    f"OpenAI api key={openai_key[0:3]}...{openai_key[-3:]}"
)

OpenAI api key=sk-...Uzy


Lets see what gpt models are available for the API key we loaded.

In [39]:
client_org = OpenAI(
    organization=openai_org_id,
    api_key=openai_key,
)
models = client_org.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
pprint(models)

[Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
 Model(id='gpt-4-0613', c

Test the direct connection to OpenAI service via OpenAI Python SDK.

In [40]:
haiku_prompt = [
    {
        "role": "system",
        "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
    },
    {
        "role": "user",
        "content": "Compose a haiku that explains the concept of recursion in programming.",
    },
]

poem_prompt = [
    {
        "role": "system",
        "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
    },
    {
        "role": "user",
        "content": "Compose a five line poem that explains the concept of recursion in programming.",
    },
]

yoda_prompt = [
    {
        "role": "system",
        "content": "Yoda assistant you are, skilled in explaining complex life and phisopical matters.",
    },
    {
        "role": "user",
        "content": "What number 42 means, be brief.",
    },
]

In [41]:
oai_client = OpenAI(api_key=openai_key)

response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=poem_prompt,
)

pprint(response.choices[0].message.content)

("In loops we dance, bound by time's decree,\n"
 "Yet recursion journeys through Infinity's sea.\n"
 'A function calls itself, a pathway untold,\n'
 'Unraveling mazes of data, brave and bold.\n'
 'A recursive embrace, where echoes of code unfold.')


Testing the streaming API

In [42]:
oai_client = OpenAI(api_key=openai_key)


response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    stream=True,
    messages=haiku_prompt
)

for chunk in response:
    print(chunk.choices[0].delta.content)




Recursive
 code
 flows
,

A
 dance
 of
 self
-reference
,

F
ragments
 build
 the
 whole
.
None


## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://localhost/


At the application start we created test projects (project1) with OpenAI API deployment. We will use project1 for this example.

In this case we will use the default `project 1` settings:
* with project_slug -> 'project1' 
* deployment_name -> 'openai'
resulting in promptsail proxy url like this: 

**http://localhost:8000/project1/openai** -> https://api.openai.com/v1

You can create your own project if you want.



In [48]:
ps_api_base = "http://localhost:8000/project1/openai"

# adress with tags 
ps_api_base = "http://localhost:8000/project1/openai/?tags=session-123456,chat,dev-joe&target_path="


ps_client = OpenAI(base_url=ps_api_base, api_key=openai_key, max_retries=0)

response = ps_client.chat.completions.create(
    model="gpt-3.5-turbo-0125", #"gpt-3.5-turbo-0125", "gpt-4-turbo-preview",
    temperature=0.5,
    messages=yoda_prompt,
)

pprint(response.choices[0].message.content)

('Meaning of life, universe, and everything, in "The Hitchhiker\'s Guide to '
 'the Galaxy" by Douglas Adams, it is.')
